# 第2单元 - 第8-9周 - @林关宁

## 1. Introduction to ANOVA (One-Way)
The analysis of variance (ANOVA) can be thought of as an extension to the t-test. 
* The independent t-test is used to compare the means of a condition between 2 groups. 
* ANOVA is used when one wants to compare the means of a condition between 2+ groups. 

ANOVA is an omnibus test, meaning it tests the data as a whole. Another way to say that is this, ANOVA tests if there is a difference in the mean somewhere in the model (testing if there was an overall effect), but it does not tell one where the difference is if the there is one. To find out where the difference is between the groups, one has to conduct post-hoc tests. This is also covered here.

* $H_0$: No difference between means, i.e. $x_1 = x_2 = x_3$
* $H_a$: Difference between means exist somewhere, i.e. $x_1 \ne x_2 \ne x_3, or  x_1 = x_2 \ne x_3, or x1 \ne x2 = x_3$

### 1.1 ANOVA Assumptions
There are 3 assumptions that need to be met for the results of an ANOVA test to be considered accurate and trust worthy. It’s important to note the the assumptions apply to the residuals and not the variables themselves. The ANOVA assumptions are the same as for linear regression and are:

1.Normality
* Caveat to this is, if group sizes are equal, the F-statistic is robust to violations of normality

2.Homogeneity of variance
* Same caveat as above, if group sizes are equal, the F-statistic is robust to this violation

3.Independent observations

If possible, it is best to have groups the same size so corrections to the data do not need to be made. However, with real world data, that is often not the case and one will have to make corrections to the data. If these assumptions are not met, and one does not want to transform the data, an alternative test that could be used is the Kruskal-Wallis H-test or Welch’s ANOVA.

# 可能需要到Anaconda去安装下：pip install researchpy 
# Researchpy produces Pandas DataFrames that contain relevant statistical testing information that is commonly required for academic research

In [1]:
import pandas as pd
import scipy.stats as stats

import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

#### Data used in this example
It is a made-up data that is measuring the effects of different doses of a clinical drug, "Difficile", on libido. It contains 2 columns of interest, “dose” and “libido”. Dose contains information on the dosing, “placebo”, “low”, and “high”, and libido is a measure of low-high libido on a 7 point Likert scale with 7 being the highest and 1 being the lowest. Now let’s import the libraries needed for the analysis, load the data and take a look! While were at it, let’s drop the ‘person’ column since we don’t need it.

这是一个伪数据组，这个数据测量不同剂量的临床药物“ Difficile”对性欲的影响。

它包含两列感兴趣的列：“剂量”和“性欲”程度。剂量包含有关剂量，“安慰剂”，“低”和“高”的信息，性欲是在7点李克特量表(Likert scale)上衡量低-高性欲，其中7最高，1最低。

现在，我们导入分析所需的库，加载数据并进行查看。在进行此操作时，我们可以去掉“人员”这列，因为我们不需要它。

In [2]:
# Loading data
df = pd.read_csv("Data_difficile.csv")
df.head(50)

,person,dose,libido
0,1,placebo,3
1,2,placebo,2
2,3,placebo,1
3,4,placebo,1
4,5,placebo,4
5,6,low,5
6,7,low,2
7,8,low,4
8,9,low,2
9,10,low,3


In [3]:
df.drop('person', axis= 1, inplace= True)
df.head()

,dose,libido
0,placebo,3
1,placebo,2
2,placebo,1
3,placebo,1
4,placebo,4


In [4]:
# Recoding value from numeric to string
#df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)
    
# Gettin summary statistics
rp.summary_cont(df['libido'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,libido,15.0,3.4667,1.7674,0.4563,2.4879,4.4454


That’s good to see the data as a whole, but we are really interested in the data by dosing.

In [5]:
rp.summary_cont(df['libido'].groupby(df['dose']))

,N,Mean,SD,SE,95% Conf.,Interval
dose,,,,,,
high,5,5.0,1.5811,0.7071,3.0368,6.9632
low,5,3.2,1.3038,0.5831,1.5811,4.8189
placebo,5,2.2,1.3038,0.5831,0.5811,3.8189


## 2，ANOVA Example
There are a few ways this can be done with Python. One is with the stats.f_oneway() method which is apart of the scipy.stats library, and the other is using statsmodels.

### 2.1. ANOVA with scipy.stats
If using scipy.stats, the method needed is stats.f_oneway(). The general applied method looks like this:

stats.f_oneway(data_group1, data_group2, data_group3, data_groupN)

In [6]:
stats.f_oneway(df['libido'][df['dose'] == 'high'], 
             df['libido'][df['dose'] == 'low'],
             df['libido'][df['dose'] == 'placebo'])

F_onewayResult(statistic=5.11864406779661, pvalue=0.024694289538222603)

The F-statistic= 5.119 and the p-value= 0.025 which is indicating that there is an overall significant effect of medication on libido. However, we don’t know where the difference between dosing/groups is yet. This is in the post-hoc section. A thing to note, is that if you are doing this for academic research purposes, this method is missing some of the information that is required for publication. For example, one would need the degrees of freedom, have to calculate the sum of squares, and conduct post-hoc tests by hand. It’s not difficult to do in Python, but there is a much easier way. Next is how to conduct an ANOVA using the regression formula; since after all, it is a generalized linear model (GLM).

F统计量= 5.119，p-值= 0.025，这表明药物对性欲有总体显著影响。但是，我们不知道剂量/组之间的差异在哪里。这是在post-hoc分析部分了。

### 2. ANOVA with statsmodels
Using statsmodels, we get a bit more information and enter the model as a regression formula. The general input using this method looks like this:

model_name = ols('outcome_variable ~ group1 + group2 + groupN', data=your_data).fit()

If you dummy code the groups, you have to not include 1 of the groups in the formula. This group’s data will still get captured in the model’s intercept and is the base (control) group. If you use the following method of entering the formula Python takes care of this for you.

model_name = ols('outcome_variable ~ C(group_variable)', data=your_data).fit()

In [8]:
results = ols('libido ~ C(dose)', data=df).fit()
aov_table = sm.stats.anova_lm(results, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(dose),20.133333,2.0,5.118644,0.024694
Residual,23.600000,12.0,NaN,NaN


Let's break down this ANOVA table. The dose row is the between groups effect which is the overall experimental effect. The sum of squares for the model (SSM; value 20.133 in the table) is how much variance is explained by our model. The current model explains a significant amount of variance, F(2,12)= 5.12, p < 0.05. The residual row is the unsystematic variation in the data (SSR; also called the unexplained variance; value 23.600 in the table). In this case, the unsystematic variation represents the natural individual differences in libido and natural different reactions to the drug, "Difficile".

让我们分解下这个方差分析表。剂量行是组间效应，是总体实验效应。模型的平方和（SSM；表中的值20.133）是我们的模型解释了多少方差。当前模型解释了很大的方差，F（2,12）= 5.12，p <0.05。错误行是数据中的非系统性变化（SSR；也称为无法解释的方差；表中值为23.600）。在这种情况下，非系统性变化代表个体本身有性欲差异和对药物Difficile的自然不同反应。

### 3. CALCULATING MODEL EFFECT SIZE 计算模型效应量大小
效果量是非常有用的一个参数。它的大小告诉我们该实验会对现实世界会有多大的影响。

有几种不同的效应量公式：eta平方（eta squared）和Ω平方（Omega squared）。与eta平方相比，Omega平方被认为是衡量效果大小的更好指标，因为它在计算中没有偏见。

需要注意的是，$R^2$在ANOVA框架内称为eta squared，他们是一样的东西。 $R^2$是模型解释多少方差的度量，并通过方差和（SSM）除以总方差（SST；也称为平方总和）来计算。总方差（SST）等于模型的平方和（SSM）加上残差的平方和（SSR）。因此，$R^2$和eta squared的公式为以下：

$R^2$ and eta squared = SS_M/SS_T

$R^2$ and eta squared = 20.133/43.733 = 0.460

这意味着当前的模型解释了性欲贡献差异的46.0％。就像刚刚提到的那样，在ANOVA框架内，R2也称为eta平方，可以解释为解释的方差量以及效应量度。

In [9]:
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(aov_table)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(dose),20.133333,2.0,10.066667,5.118644,0.024694,0.460366,0.354486
Residual,23.600000,12.0,1.966667,NaN,NaN,NaN,NaN


### 4. Assumption Checks/Model Diagnostics 假设检查/模型诊断
#### 4.1 Assumption: Homogeneity of Variance 方差齐性

可以使用Levene检验来检验组间的方差齐性。这是scipy.stats库的一部分。对于分类变量的每个层次，都应该检查方差假设的同质性。

当Levene的方差齐性检验为不显著时，才表明两组具有相同的方差。

In [9]:
stats.levene(df['libido'][df['dose'] == 'placebo'],
             df['libido'][df['dose'] == 'low'],
             df['libido'][df['dose'] == 'high'])

LeveneResult(statistic=0.11764705882352934, pvalue=0.8900225182757423)

#### 4.2 Assumption: Normality 正态性

我们可以使用提供的Jarque-Bera测试，也可以使用Shapiro或其他测试。我将演示如何使用Shapiro方法测试正态性。输出没有标记，但数字是测试统计值，后跟p值。

Shapiro-Wilk检验的结果如果是不显著的，没有统计学意义，这表明是正态分布的。

In [40]:
stats.shapiro(results.resid)

(0.9166916012763977, 0.17146942019462585)

### 5. Post-hoc Testing 事后分析（两两样本之间的多重比较）
The overall model was significant, now to test which groups differ. Deciding which groups to compare should be theory driven. There are a few different techniques that can be used. Each of these techniques have different ways of controlling for familywise error rate. 3 common methods are:

如果上面的分析是说整个模型是有显著性的，那就需要来测试具体哪些组是不同。可以使用一些不同的方法来控制误差率的方法。例如2种常用方法是：


#### 5.1 BONFERRONI CORRECTION POST-HOC COMPARISON
First the corrected p-value needs to be calculated. This can be done using the formula:

p-value/# of comparisons = 0.05/3 = 0.01667

Now the t-tests that are conducted have to have a p-value less than 0.01667 in order to be considered significant.

In [47]:
stats.ttest_ind(df['libido'][df['dose'] == 'high'], df['libido'][df['dose'] == 'low'])

Ttest_indResult(statistic=1.963961012123931, pvalue=0.08513507177899203)

In [45]:
stats.ttest_ind(df['libido'][df['dose'] == 'low'], df['libido'][df['dose'] == 'placebo'])

Ttest_indResult(statistic=1.2126781251816647, pvalue=0.2598450452137845)

In [48]:
stats.ttest_ind(df['libido'][df['dose'] == 'high'], df['libido'][df['dose'] == 'placebo'])

Ttest_indResult(statistic=3.0550504633038926, pvalue=0.015700141250047695)

Using the Bonferroni correction, only the difference between the high dose and placebo groups are significantly different. We can calculate the high dosing’s effect size! To calculate the effect size for the treatment dosing we also need to calculate the degrees of freedom since it’s not provided. The following equations can be used:
使用Bonferroni校正，只有高剂量组和安慰剂组之间的差异显著不同。我们可以计算高剂量的效应大小！为了计算治疗剂量的效应大小，我们还需要计算自由度，因为它没有提供。可以使用以下方程式：

* dof = #_observations_group1 + #_observations_group2 - #_of_groups
* dof = 5 + 5 - 2 = 8
* effect size r = square root of ($t^2$/$t^2$ + dof)
* effect size r = sqrt(1.213**2/(1.213**2 + 8)) = 0.39

The high dose has a medium effect size.

#### 5.2 TUKEY’S HSD POST-HOC COMPARISON
Tukey's HSD: Method also controls for familywise error rate (FWER) and is also considered conservative.

Tukey HSD事后比较测试控制 I型错误，并将家族错误率保持在0.05（FWER=0.05，在表顶部）。group1和group2列是被比较的组，meandiff列是被计算为group2–group1的两个组的平均值的差异，下/上列是95%置信区间的下/上边界，拒绝列说明是否应拒绝零假设。可信的是，这种方法目前没有提供t统计量，因此无法计算治疗效应量大小。

In [41]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df['libido'], df['dose'])
mc_results = mc.tukeyhsd()
print(mc_results)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1  group2 meandiff p-adj   lower   upper  reject
-----------------------------------------------------
  high     low     -1.8 0.1472 -4.1651  0.5651  False
  high placebo     -2.8 0.0209 -5.1651 -0.4349   True
   low placebo     -1.0 0.5171 -3.3651  1.3651  False
-----------------------------------------------------


### 6. ANOVA Results Interpretation
While interpreting the ANOVA results, the Bonferroni post-hoc analysis results will be used here.

There was a significant effect of Difficile on the level of libido, F(2,12)= 5.12, p < 0.05, omega_sq = 0.35. Planned post-hoc testing, using the Bonferroni correction P= 0.0167, revealed that high dose of Difficile significantly increased libido compared to the placebo, t(8)=3.06, p < 0.0167, r= 0.39. There were no other statistically significant differences between groups.

## 3. Repeated measure ANOVA (One-Way & Two-way)

This code examples show you how to carry out a repeated measures ANOVA using Statsmodels AnovaRM. There are also more Python ANOVA guides found on https://www.marsja.se

In [8]:
import pandas as pd
from statsmodels.stats.anova import AnovaRM

# help(AnovaRM)

### One-way repeated measure ANOVA

In [9]:
df = pd.read_csv('Data_rmAOV1way.csv')

aovrm = AnovaRM(df, 'rt', 'Sub_id', within=['cond'])
res = aovrm.fit()

print(res)

               Anova
     F Value  Num DF  Den DF Pr > F
-----------------------------------
cond 499.1549 1.0000 59.0000 0.0000



### Two-way repeated measure ANOVA

In [11]:
df = pd.read_csv('Data_rmAOV2way.csv')
df.head()

,Sub_id,rt,iv1,iv2
0,1,1082.986553,noise,down
1,2,938.799689,noise,down
2,3,1101.470970,noise,down
3,4,1123.030275,noise,down
4,5,938.051589,noise,down


In [10]:
df2way = pd.read_csv('Data_rmAOV2way.csv')
aovrm2way = AnovaRM(df2way, 'rt', 'Sub_id', within=['iv1', 'iv2'])
res2way = aovrm2way.fit()

print(res2way)

                 Anova
         F Value  Num DF  Den DF  Pr > F
----------------------------------------
iv1     2207.0162 1.0000  59.0000 0.0000
iv2      275.4144 2.0000 118.0000 0.0000
iv1:iv2    1.8651 2.0000 118.0000 0.1594

